<a href="https://colab.research.google.com/github/lazy-wav/MusicGeneration_Tabla/blob/main/MusicGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install librosa
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d pranav6670/tabla-taala-dataset

Dataset URL: https://www.kaggle.com/datasets/pranav6670/tabla-taala-dataset
License(s): CC0-1.0
 99% 855M/860M [00:18<00:00, 54.5MB/s]
100% 860M/860M [00:18<00:00, 47.7MB/s]


In [ ]:
# extracting the data zipped file
from zipfile import ZipFile
dataset='/content/tabla-taala-dataset.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print ('Dataset extracted successfully!')

Dataset extracted successfully!


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler
from google.colab import files

In [ ]:
import io

In [ ]:
csv_path = '/content/file_name.csv'
df = pd.read_csv(csv_path)
df.head()

,fname,label
0,addhatrital01.wav,Addhatrital
1,addhatrital02.wav,Addhatrital
2,addhatrital03.wav,Addhatrital
3,addhatrital04.wav,Addhatrital
4,addhatrital05.wav,Addhatrital


In [ ]:
def extract_features(file_name):
    y, sr = librosa.load(file_name, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    return np.mean(mfccs.T, axis=0)

features = []
labels = []

for index, row in df.iterrows():
    file_path = '/content/wavfiles/' + row['fname']
    label = row['label']
    features.append(extract_features(file_path))
    labels.append(label)

features = np.array(features)
labels = np.array(labels)

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.utils import to_categorical

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)

# Now convert integer labels to one-hot encoding
labels = to_categorical(integer_encoded)
# labels = to_categorical(labels)

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(scaled_features, labels, test_size=0.2, random_state=42)

# Reshape for LSTM input
x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))


In [ ]:
model = Sequential()

model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.2))

model.add(LSTM(units=60, activation='relu', return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(units=80, activation='relu', return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(units=120, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=y_train.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_test, y_test))


Epoch 1/50
14/14 [==============================] - 6s 61ms/step - loss: 2.0793 - accuracy: 0.1562 - val_loss: 2.0784 - val_accuracy: 0.1770
Epoch 2/50
14/14 [==============================] - 0s 11ms/step - loss: 2.0780 - accuracy: 0.1763 - val_loss: 2.0773 - val_accuracy: 0.1770
Epoch 3/50
14/14 [==============================] - 0s 9ms/step - loss: 2.0768 - accuracy: 0.1763 - val_loss: 2.0756 - val_accuracy: 0.1770
Epoch 4/50
14/14 [==============================] - 0s 9ms/step - loss: 2.0732 - accuracy: 0.1763 - val_loss: 2.0717 - val_accuracy: 0.1770
Epoch 5/50
14/14 [==============================] - 0s 9ms/step - loss: 2.0472 - accuracy: 0.1763 - val_loss: 2.0390 - val_accuracy: 0.1770
Epoch 6/50
14/14 [==============================] - 0s 8ms/step - loss: 1.9403 - accuracy: 0.1763 - val_loss: 1.9898 - val_accuracy: 0.1770
Epoch 7/50
14/14 [==============================] - 0s 11ms/step - loss: 1.8573 - accuracy: 0.1853 - val_loss: 1.9021 - val_accuracy: 0.2035
Epoch 8/50
14/14 

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {accuracy}')

4/4 [==============================] - 0s 7ms/step - loss: 0.5313 - accuracy: 0.8142
Test Accuracy: 0.8141592741012573


In [ ]:
model.save('music_generation_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# from keras.models import load_model

# model = load_model('music_generation_model.h5')

In [ ]:
seed_sequence = features[0]
seed_sequence = np.expand_dims(seed_sequence, axis=0)

In [ ]:
def generate_sequence(model, seed, sequence_length):
    generated_sequence = []

    current_sequence = seed
    for _ in range(sequence_length):
        prediction = model.predict(current_sequence)  # prediction has shape (1, 40)
        generated_sequence.append(prediction)

        # Reshape prediction to (1, 1, 40) and concatenate along sequence length dimension
        reshaped_prediction = np.reshape(prediction, (prediction.shape[0], 1, prediction.shape[1]))
        current_sequence = np.concatenate([current_sequence[:, 1:, :], reshaped_prediction], axis=1)

    return np.array(generated_sequence)

In [ ]:
print(seed_sequence.shape)

(1, 40)


In [ ]:
import numpy as np
from keras.models import load_model

# Load the trained model
model = load_model('music_generation_model.h5')

# Print model summary to understand input and output shapes
model.summary()

# Example: Dummy seed sequence for illustration (replace with actual data handling)
seed_sequence = np.random.rand(1, 40)  # Example shape (1, 40)

# Reshape seed_sequence to match expected model input shape (batch_size, timesteps, input_dim)
seed_sequence = seed_sequence.reshape(1, 1, 40)  # Adjust input_dim as necessary

# Function to generate a sequence from the seed
def generate_sequence(model, seed, sequence_length):
    generated_sequence = []

    current_sequence = seed
    for _ in range(sequence_length):
        # Print current sequence shape for debugging
        print(f'Current sequence shape: {current_sequence.shape}')

        # Predict the next value
        prediction = model.predict(current_sequence)

        # Print prediction shape for debugging
        print(f'Prediction shape: {prediction.shape}')

        generated_sequence.append(prediction[0])

        # Ensure prediction shape matches the input shape (1, 1, 40)
        prediction = np.pad(prediction, ((0, 0), (0, 40 - prediction.shape[1])), 'constant')

        # Update current_sequence to include the predicted value
        # Keep only the last 39 timesteps and append the new prediction
        current_sequence = np.concatenate((current_sequence[:, 1:, :], prediction.reshape(1, 1, 40)), axis=1)  # Concatenate along the timesteps axis (axis=1)

    return np.array(generated_sequence)

# Generate a sequence of 100 timesteps
generated_sequence = generate_sequence(model, seed_sequence, 2000)

print("Generated sequence shape:", generated_sequence.shape)


Streaming output truncated to the last 5000 lines.
Prediction shape: (1, 8)
Current sequence shape: (1, 1, 40)
1/1 [==============================] - 0s 19ms/step
Prediction shape: (1, 8)
Current sequence shape: (1, 1, 40)
1/1 [==============================] - 0s 20ms/step
Prediction shape: (1, 8)
Current sequence shape: (1, 1, 40)
1/1 [==============================] - 0s 20ms/step
Prediction shape: (1, 8)
Current sequence shape: (1, 1, 40)
1/1 [==============================] - 0s 20ms/step
Prediction shape: (1, 8)
Current sequence shape: (1, 1, 40)
1/1 [==============================] - 0s 20ms/step
Prediction shape: (1, 8)
Current sequence shape: (1, 1, 40)
1/1 [==============================] - 0s 22ms/step
Prediction shape: (1, 8)
Current sequence shape: (1, 1, 40)
1/1 [==============================] - 0s 31ms/step
Prediction shape: (1, 8)
Current sequence shape: (1, 1, 40)
1/1 [==============================] - 0s 25ms/step
Prediction shape: (1, 8)
Current sequence shape: (1, 

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 50)             18200     
                                                                 
 dropout (Dropout)           (None, 1, 50)             0         
                                                                 
 lstm_1 (LSTM)               (None, 1, 60)             26640     
                                                                 
 dropout_1 (Dropout)         (None, 1, 60)             0         
                                                                 
 lstm_2 (LSTM)               (None, 1, 80)             45120     
                                                                 
 dropout_2 (Dropout)         (None, 1, 80)             0         
                                                                 
 lstm_3 (LSTM)               (None, 120)               9

In [ ]:
!pip install soundfile
import numpy as np
import librosa
import soundfile as sf

# Function to convert MFCCs to audio waveform
def mfcc_to_audio(mfcc_sequence, sample_rate=22050):
    # Invert MFCCs to get audio waveform
    waveform = librosa.feature.inverse.mfcc_to_audio(mfcc_sequence.T)
    return waveform

# Example usage:
# Assuming generated_sequence contains your generated MFCC sequences
generated_audio = mfcc_to_audio(generated_sequence)

# Save the generated audio to a file using soundfile
sf.write('generated_music5.wav', generated_audio, samplerate=22050)

print("Generated music saved as generated_music5.wav")


Generated music saved as generated_music5.wav
